In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import aplpy
from astropy.io import fits
from astropy import wcs
import warnings
#import regions
#import pyregion
from astropy.utils.console import ProgressBar
from astropy.coordinates import SkyCoord

In [2]:

def cutout(inputfile, xcenter, ycenter, width, height, outfile, cdelt = ''):

    ### This function creates a new fits file based on rectangular region information.
    ### xcenter, ycenter are the centers of the region in galactic longitude and latitude in degrees
    ### width and height are similarly longitude and latitude measurements in degrees
    
    w = wcs.WCS(inputfile).celestial
    print(w)
    data, header = fits.getdata(inputfile, header = True)
    print(data.shape)
    data = data[0,:,:]
    print(data.shape)
    #if (header['CRVAL1'] != 0) or (header['CRVAL2'] != 0):
    #    raise ValueError('\nThe input CRVAL is not 0!\n')
    
    center_x, center_y = w.wcs_world2pix(xcenter, ycenter, 1)
    print(center_x,center_y)
    center_x = int(center_x)
    center_y = int(center_y)
    
    if cdelt == '':
        cdelt = np.mean( [np.absolute(header['CDELT1']), np.absolute(header['CDELT2'])] )
        
    size_pixels_x = int(np.rint((width / 2.) / cdelt))
    size_pixels_y = int(np.rint((height / 2.) / cdelt))
    
    delta_y = np.array([center_x - size_pixels_x, center_x + size_pixels_x], dtype = int)
    delta_x = np.array([center_y - size_pixels_y, center_y + size_pixels_y], dtype = int)
    
    center_x_new, center_y_new  = w.wcs_pix2world( delta_y[0], delta_x[0], 1)
    
    data_cutout = data[delta_x[0]:delta_x[1], delta_y[0]:delta_y[1]]

    center_x = ((delta_x[1] - delta_x[0]) / 2.) + delta_x[0]
    center_y = ((delta_y[1] - delta_y[0]) / 2.) + delta_y[0]
    
    del header[33:] ### strictly for FORCAST files, needs to be 28 for cmzoom mosaic I think...
    del header['PC*']
        
    header['CRPIX1'] = size_pixels_x
    header['CRPIX2'] = size_pixels_y
    header['CRVAL1'] = xcenter
    header['CRVAL2'] = ycenter
    
    fits.writeto(outfile, data = data_cutout, header = header, overwrite = True)
    
def make_single(inputfits, outputfile, contourfile, xcenter, ycenter, label, vmin=1.0e06, vmax =3.0e+08):
    
    fg_color='white'
    bg_color='black'
    fig = plt.figure(figsize=(4, 4))#,facecolor=bg_color, edgecolor=fg_color)
    subplot = aplpy.FITSFigure(inputfits, figure = fig, convention='calabretta')
    subplot.show_colorscale(vmin=vmin, vmax=vmax, stretch='log',cmap='inferno')
    #subplot.frame.set_color(fg_color)
 
    subplot.set_nan_color(fg_color)

    #subplot.ticks.set_xspacing(0.02)
    #subplot.ticks.set_yspacing(0.02)
    subplot.ticks.set_color('black')
    subplot.tick_labels.set_xformat('d.dd')
    subplot.tick_labels.set_yformat('d.dd')
    #subplot.recenter(xcenter, ycenter, width = 50. / 3600., height = 50. / 3600.)
    subplot.add_label(0.35, 0.95, str(label), relative = True, weight = 'bold', size = 10, color = 'black')

    subplot.show_contour(contourfile,
                         colors = 'white', levels = [0.0,1.0], linewidths = 0.7,
                         convention = 'calabretta', zorder = 10, linestyle = 'solid')
    subplot.axis_labels.set_xtext('Galactic Longitude (degrees)')
    subplot.axis_labels.set_ytext('Galactic Latitude (degrees)')
    subplot.ticks.show()
    #subplot.add_scalebar(length=24./3600.)
    #subplot.scalebar.set_label('1 pc')
    #subplot.scalebar.set_color('k')
    fig.tight_layout()
    fig.savefig(outputfile)
    plt.show()

        

In [3]:
### save four test cutouts for SOFIA special session talk

savepath = '/Users/hph/Dropbox/astrophys/IGNITES/cutouts/'
datapath = '/Users/hph/Dropbox/astrophys/IGNITES/data/FORCAST_Files/'
mosaic_file = datapath+'F25_cmzoom_regrid.fits'

### for G0.619
xcen, ycen = 0.6, -0.05
width, height = 0.06, 0.06
#sc = SkyCoord([(xcen,ycen)], frame='galactic',unit='deg')
#xcen_ra, ycen_dec = sc.icrs.ra.degree[0], sc.icrs.dec.degree[0]

cutout(mosaic_file,
       xcen, ycen, width, height,
       outfile=savepath+'G0.619_F25.fits')

### for sitcks and stones
xcen, ycen = 0.08, -0.08
width, height = 0.06, 0.06
#sc = SkyCoord([(xcen,ycen)], frame='galactic',unit='deg')
#xcen_ra, ycen_dec = sc.icrs.ra.degree[0], sc.icrs.dec.degree[0]

cutout(mosaic_file,
       xcen, ycen, width, height,
       outfile=savepath+'sticks_stones_F25.fits')

### for 20 km/s cloud
xcen, ycen = 359.97, -0.077
width, height = 0.06, 0.06
#sc = SkyCoord([(xcen,ycen)], frame='galactic',unit='deg')
#xcen_ra, ycen_dec = sc.icrs.ra.degree[0], sc.icrs.dec.degree[0]

cutout(mosaic_file,
       xcen, ycen, width, height,
       outfile=savepath+'20kms_F25.fits')

### for G0.619
xcen, ycen = 0.101, -0.005
width, height = 0.06, 0.06
#sc = SkyCoord([(xcen,ycen)], frame='galactic',unit='deg')
#xcen_ra, ycen_dec = sc.icrs.ra.degree[0], sc.icrs.dec.degree[0]

cutout(mosaic_file,
       xcen, ycen, width, height,
       outfile=savepath+'G0.101_F25.fits')


WCS Keywords

Number of WCS axes: 2
CTYPE : 'GLON-CAR'  'GLAT-CAR'  
CRVAL : 0.418  -0.081  
CRPIX : 9500.0  1400.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : -0.00013888888888  0.00013888888888  
NAXIS : 19000  2800
(3, 2800, 19000)
(2800, 19000)
8189.600307084119 1623.1970577315672
WCS Keywords

Number of WCS axes: 2
CTYPE : 'GLON-CAR'  'GLAT-CAR'  
CRVAL : 0.418  -0.081  
CRPIX : 9500.0  1400.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : -0.00013888888888  0.00013888888888  
NAXIS : 19000  2800
(3, 2800, 19000)
(2800, 19000)
11933.597628283995 1407.1898521555179
WCS Keywords

Number of WCS axes: 2
CTYPE : 'GLON-CAR'  'GLAT-CAR'  
CRVAL : 0.418  -0.081  
CRPIX : 9500.0  1400.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : -0.00013888888888  0.00013888888888  
NAXIS : 19000  2800
(3, 2800, 19000)
(2800, 19000)
12725.597095158155 1428.7821722930546
WCS Keywords

Number of WCS axes: 2
CTYPE : 'GLON-CAR'  'GLAT-CAR'  
CRVAL : 0.418  -0